## **Load data from Bronze layer to Silver layer**

In [1]:
silver_df_master = spark.read.csv("Files/data/master_dataframe.csv",header=True,inferSchema=True)
silver_df_iplraw = spark.read.csv("Files/data/ipl_summary_raw.csv",header=True,inferSchema=True)
silver_df_iplballbyball = spark.read.csv("Files/data/ipl_ball_by_ball_output_raw.csv",header=True,inferSchema=True)

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 3, Finished, Available, Finished)

### **Dataset-Iplraw**

In [8]:
display(silver_df_iplraw)

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a1debb3a-c5ea-498b-a3dd-6e58c5f02a16)

#### **Streamline venue names by removing city names from the venue**

In [6]:
from pyspark.sql.functions import col,regexp_replace
silver_df_iplraw_cleaned = silver_df_iplraw.withColumn(
    "info_venue",
    regexp_replace(col("info_venue"),".,*$","")
) 
silver_df_iplraw_cleaned.select("info_venue").distinct().show()

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 8, Finished, Available, Finished)

+--------------------+
|          info_venue|
+--------------------+
|M Chinnaswamy Sta...|
|Sardar Patel Stad...|
|Wankhede Stadium,...|
|     Wankhede Stadiu|
|Sawai Mansingh St...|
|Narendra Modi Sta...|
|Bharat Ratna Shri...|
|Dubai Internation...|
|MA Chidambaram St...|
|MA Chidambaram St...|
|Dr DY Patil Sport...|
|Barsapara Cricket...|
|      OUTsurance Ova|
|Vidarbha Cricket ...|
|Brabourne Stadium...|
| Arun Jaitley Stadiu|
|Himachal Pradesh ...|
|Rajiv Gandhi Inte...|
|Maharaja Yadavind...|
|Rajiv Gandhi Inte...|
+--------------------+
only showing top 20 rows



In [7]:
display(silver_df_iplraw_cleaned)

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5cc0478b-2127-4ab9-9823-e067b3a705de)

In [10]:
from pyspark.sql.functions import col, when

silver_df_iplraw_cleaned = silver_df_iplraw.withColumn(
    "info_city", when(col("info_city") == "Bangalore", "Bengaluru")
               .otherwise(col("info_city"))
).withColumn(
    "info_teams_1", when(col("info_teams_1") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("info_teams_1") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("info_teams_1"))
).withColumn(
    "info_teams_2", when(col("info_teams_2") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("info_teams_2") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("info_teams_2"))
).withColumn(
    "info_toss_winner", when(col("info_toss_winner") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("info_toss_winner") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("info_toss_winner"))      
).withColumn(
    "info_outcome_winner", when(col("info_outcome_winner") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("info_outcome_winner") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("info_outcome_winner"))            
).withColumn(
    "info_venue", when(col("info_venue") == "M.Chinnaswamy Stadium", "M Chinnaswamy Stadium")
             .otherwise(col("info_venue"))
).na.fill("Unknown", subset=["info_venue", "info_city", "info_teams_1", "info_teams_2", "info_toss_winner","info_outcome_winner"])

silver_df_iplraw_cleaned.select("info_venue").distinct().show()

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 12, Finished, Available, Finished)

+--------------------+
|          info_venue|
+--------------------+
|Dubai Internation...|
|Himachal Pradesh ...|
|M Chinnaswamy Sta...|
|Sardar Patel Stad...|
|Punjab Cricket As...|
|Punjab Cricket As...|
|    Barabati Stadium|
|Punjab Cricket As...|
|       Nehru Stadium|
|Maharashtra Crick...|
|Brabourne Stadium...|
|        Eden Gardens|
|Arun Jaitley Stad...|
|     OUTsurance Oval|
|M Chinnaswamy Sta...|
|Punjab Cricket As...|
|    Feroz Shah Kotla|
|Sawai Mansingh St...|
|Rajiv Gandhi Inte...|
|Bharat Ratna Shri...|
+--------------------+
only showing top 20 rows



In [11]:
display(silver_df_iplraw_cleaned)

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 13d45cd9-1eae-4973-adcf-4aa0c65caa73)

In [12]:
from pyspark.sql.functions import regexp_replace, trim

# Clean venue names by removing city or extra descriptions after commas
# We want to keep only the main venue name, even if there are commas or additional details
silver_df_iplraw_cleaned = silver_df_iplraw.withColumn(
    "info_venue",
    trim(
        regexp_replace(col("info_venue"), r",.*$", "")
    )
)

# Manually handle cases where venues might still need extra corrections
# For example, handle specific cases like "MA Chidambaram Stadium" without the city part
venue_cleaning_rules = [
    (r"Rajiv Gandhi International Stadium.*", "Rajiv Gandhi International Stadium"),
    (r"Punjab Cricket Association IS Bindra Stadium.*", "Punjab Cricket Association IS Bindra Stadium"),
    (r"MA Chidambaram Stadium.*", "MA Chidambaram Stadium"),
    # Add any other complex venues here
]

# Apply the specific cleaning rules
for pattern, replacement in venue_cleaning_rules:
    silver_df_iplraw_cleaned = silver_df_iplraw_cleaned.withColumn(
        "info_venue",
        regexp_replace(col("info_venue"), pattern, replacement)
    )

# Show the cleaned venue names to verify
silver_df_iplraw_cleaned.select("info_venue").distinct().show()

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 14, Finished, Available, Finished)

+--------------------+
|          info_venue|
+--------------------+
|Dubai Internation...|
|Himachal Pradesh ...|
|Punjab Cricket As...|
|Vidarbha Cricket ...|
|    Barabati Stadium|
|       Nehru Stadium|
|Maharashtra Crick...|
|        Eden Gardens|
|     OUTsurance Oval|
|M Chinnaswamy Sta...|
|    Feroz Shah Kotla|
|   Brabourne Stadium|
|Barsapara Cricket...|
|M.Chinnaswamy Sta...|
|          Green Park|
|Holkar Cricket St...|
|Bharat Ratna Shri...|
|Shaheed Veer Nara...|
|Sheikh Zayed Stadium|
|Sharjah Cricket S...|
+--------------------+
only showing top 20 rows



In [13]:
from delta.tables import DeltaTable
silver_table = DeltaTable.createIfNotExists(spark).location("Tables/Silver_iplrawdata_table").execute()

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 15, Finished, Available, Finished)

In [16]:
silver_df_iplraw_cleaned.write.option("overwriteschema","true").format("delta").mode("overwrite").save("Tables/Silver_iplrawdata_table")

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 18, Finished, Available, Finished)

#### **Dataset-master**

In [18]:

from pyspark.sql.functions import regexp_replace, trim, col

silver_df_master_cleaned = silver_df_master.withColumn(
    "venue_cleaned",
    trim(
         regexp_replace(col("venue"), r"([a-z])([A-Z])", r"$1 $2")
    )
)
silver_df_master_cleaned.select("venue", "venue_cleaned").distinct().show()

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 20, Finished, Available, Finished)

+--------------------+--------------------+
|               venue|       venue_cleaned|
+--------------------+--------------------+
|                null|                null|
|      SuperSportPark|    Super Sport Park|
| NewWanderersStadium|New Wanderers Sta...|
|      StGeorge'sPark|    St George's Park|
|         EdenGardens|        Eden Gardens|
|    BrabourneStadium|   Brabourne Stadium|
|            Newlands|            Newlands|
|SawaiMansinghStadium|Sawai Mansingh St...|
|DrDYPatilSportsAc...|Dr DYPatil Sports...|
|     BarabatiStadium|    Barabati Stadium|
|  SardarPatelStadium|Sardar Patel Stadium|
|  DeBeersDiamondOval|De Beers Diamond ...|
|      OUTsuranceOval|     OUTsurance Oval|
|MAChidambaramStadium|MAChidambaram Sta...|
|     WankhedeStadium|    Wankhede Stadium|
| MChinnaswamyStadium|MChinnaswamy Stadium|
|         BuffaloPark|        Buffalo Park|
|      FerozShahKotla|    Feroz Shah Kotla|
|RajivGandhiIntern...|Rajiv Gandhi Inte...|
|PunjabCricketAsso...|Punjab Cri

In [19]:
from pyspark.sql.functions import col, when


sliver_df_master_cleaned = silver_df_master.withColumn(
    "city", when(col("city") == "Bangalore", "Bengaluru")
               .otherwise(col("city"))
).withColumn(
    "team_1", when(col("team_1") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("team_1") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("team_1"))
).withColumn(
    "team_2", when(col("team_2") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("team_2") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("team_2"))
).withColumn(
    "toss_winner", when(col("toss_winner") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("toss_winner") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("toss_winner"))      
).withColumn(
    "match_winner", when(col("match_winner") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("match_winner") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("match_winner"))            
).withColumn(
    "venue", when(col("venue") == "M.Chinnaswamy Stadium", "M Chinnaswamy Stadium")
    .when(col("venue") == "MChinnaswamy Stadium", "M Chinnaswamy Stadium")
    .when(col("venue") == "MAChidambaram Stadium", "MA Chidambaram Stadium")
             .otherwise(col("venue"))
).na.fill("Unknown", subset=["venue", "city", "team_1", "team_2", "toss_winner","match_winner"])

silver_df_master_cleaned.select("venue","city", "team_1", "team_2", "toss_winner","match_winner").distinct().show()
     

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 21, Finished, Available, Finished)

+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|               venue|          city|              team_1|              team_2|         toss_winner|        match_winner|
+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|           Kingsmead|        Durban|    Rajasthan Royals|     Kings XI Punjab|     Kings XI Punjab|    Rajasthan Royals|
|      SuperSportPark|     Centurion| Chennai Super Kings|    Rajasthan Royals|    Rajasthan Royals| Chennai Super Kings|
|SawaiMansinghStadium|        Jaipur|    Rajasthan Royals|Kolkata Knight Ri...|    Rajasthan Royals|    Rajasthan Royals|
| NewWanderersStadium|  Johannesburg|     Deccan Chargers|Royal Challengers...|Royal Challengers...|     Deccan Chargers|
|  DeBeersDiamondOval|     Kimberley|     Deccan Chargers|    Rajasthan Royals|     Deccan Chargers|     Deccan Chargers|
|PunjabCricketAsso...|  

In [20]:
from delta import DeltaTable
silver_master_table = DeltaTable.createIfNotExists(spark).location("Tables/Silver_master_table").execute()

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 22, Finished, Available, Finished)

In [21]:
silver_df_master_cleaned.write.option("overwriteschema","true").format("delta").mode("overwrite").save("Tables/Silver_master_table")

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 23, Finished, Available, Finished)

#### **Dataset-ballbyball**

In [23]:
display(silver_df_iplballbyball)

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5961b0b9-1c03-4314-87da-2b5db15fe1e4)

In [29]:
from pyspark.sql.functions import col, when


silver_df_iplballbyball_cleaned = silver_df_iplballbyball.withColumn(
    "batter", when(col("batter") == "BB McCullum", "Brendon McCullum")
               .otherwise(col("batter")))
silver_df_iplballbyball_cleaned.select("batter").distinct().show()

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 30, Finished, Available, Finished)

+---------------+
|         batter|
+---------------+
|     S Anirudha|
|     TM Dilshan|
| M Muralitharan|
|  LA Carseldine|
|        J Botha|
|     KA Pollard|
|       DR Smith|
| Jaskaran Singh|
|     A Flintoff|
|       M Manhas|
|      GR Napier|
|          B Lee|
|       A Mukund|
|     D du Preez|
|Mohammad Hafeez|
|      LPC Silva|
|     AL Menaria|
|Joginder Sharma|
|KB Arun Karthik|
|      YK Pathan|
+---------------+
only showing top 20 rows



In [30]:
display(silver_df_iplballbyball_cleaned)

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ebbd5677-b37b-4110-8b49-f64d058a0f77)

In [31]:
from delta import DeltaTable
silver_iplballbyball_table = DeltaTable.createIfNotExists(spark).location("Tables/Silver_iplballbyball_table").execute()

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 32, Finished, Available, Finished)

In [32]:
silver_df_iplballbyball_cleaned.write.option("overwriteschema","true").format("delta").mode("overwrite").save("Tables/Silver_iplballbyball_table")

StatementMeta(, c58df2c7-822d-4eed-b6e7-26cac127e755, 33, Finished, Available, Finished)